In [1]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense, MaxPooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.test.is_gpu_available()

True

In [3]:
batchsize = 8

In [4]:
# r'C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\Dataset\Train'

In [5]:
# r'C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\Dataset\Train'

In [6]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)
train_data = train_datagen.flow_from_directory(r'C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\Dataset\Train',
                                               target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='training')
validation_data = train_datagen.flow_from_directory(r'C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\Dataset\Train', 
                                                target_size=(80,80), batch_size=batchsize, class_mode='categorical', subset='validation')

Found 61113 images belonging to 2 classes.
Found 15277 images belonging to 2 classes.


In [7]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(r'C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\Dataset\Test',
                                             target_size=(80,80), batch_size=batchsize, class_mode='categorical')

Found 8508 images belonging to 2 classes.


In [7]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2, activation='softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable = False

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 80, 80, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 39, 39, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 39, 39, 32)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 39, 39, 32)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [9]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
checkpoint = ModelCheckpoint(r'C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\models\model.h5', 
                             monitor='val_loss', save_best_only=True, verbose=3)

earlystop = EarlyStopping(monitor='val_loss', patience=7, verbose=3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=3)

callbacks = [checkpoint, earlystop, learning_rate]

In [10]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit_generator(train_data, steps_per_epoch=train_data.samples//batchsize,
                    validation_data=validation_data,
                    validation_steps = validation_data.samples//batchsize, 
                    callbacks=callbacks, 
                    epochs=20)

Epoch 1/20
1908/7639 [======>.......................] - ETA: 2:49 - loss: 1.4166 - acc: 0.6274
Epoch 00001: val_loss improved from inf to 1.41691, saving model to C:\Programming\ML\Drivers Drowsiness Detection using Deep Learning\models\model.h5
7639/7639 [==============================] - 306s 40ms/step - loss: 0.4784 - acc: 0.7678 - val_loss: 1.4169 - val_acc: 0.6274
Epoch 2/20
1909/7639 [======>.......................] - ETA: 2:16 - loss: 2.1789 - acc: 0.5776
Epoch 00002: val_loss did not improve from 1.41691
7639/7639 [==============================] - 245s 32ms/step - loss: 0.4438 - acc: 0.7921 - val_loss: 2.1789 - val_acc: 0.5776
Epoch 3/20
1909/7639 [======>.......................] - ETA: 2:17 - loss: 1.8829 - acc: 0.5999
Epoch 00003: val_loss did not improve from 1.41691
7639/7639 [==============================] - 241s 32ms/step - loss: 0.4374 - acc: 0.7941 - val_loss: 1.8829 - val_acc: 0.5999
Epoch 4/20
1908/7639 [======>.......................] - ETA: 2:33 - loss: 2.2340 - a

In [ ]:
# Model evaluation

In [ ]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_ts, loss_ts = model.evaluate(test_data)
print(acc_ts)
print(loss_ts)